-sandbox
# Machine Learning Pipeline

** What you will learn:**
* How to create a Machine Learning Pipeline.
* How to train a Machine Learning model.
* How to save & read the model.
* How to make predictions with the model.

## Mount data

In [3]:
%run "../includes/mnt_blob"

## The Data

The dataset contains bike rental info from 2011 and 2012 in the Capital bikeshare system, plus additional relevant information such as weather.  

This dataset is from Fanaee-T and Gama (2013) and is hosted by the <a href="http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset" target="_blank">UCI Machine Learning Repository</a>.

## The Goal
We want to learn to predict bike rental counts (per hour) from information such as day of the week, weather, month, etc.  

Having good predictions of customer demand allows a business or service to prepare and increase supply as needed.

## Loading the data

We begin by loading our data, which is stored in the CSV format</a>.

In [7]:
fileName = "/mnt/data/bikeSharing/data-001/hour.csv"

initialDF = (spark.read          # Our DataFrameReader
  .option("header", "true")      # Let Spark know we have a header
  .option("inferSchema", "true") # Infering the schema (it is a small dataset)
  .csv(fileName)                 # Location of our data
  .cache()                       # Mark the DataFrame as cached.
)

initialDF.count()                # Materialize the cache

Out[ 4 ]: 17379

In [8]:
initialDF.printSchema()

root
-- instant: integer (nullable = true)
-- dteday: timestamp (nullable = true)
-- season: integer (nullable = true)
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- holiday: integer (nullable = true)
-- weekday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- cnt: integer (nullable = true)

## Understanding the data

According to the <a href="http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset" target="_blank">UCI ML Repository description</a>, we have the following schema:

**Feature columns**:
* **dteday**: date
* **season**: season (1:spring, 2:summer, 3:fall, 4:winter)
* **yr**: year (0:2011, 1:2012)
* **mnth**: month (1 to 12)
* **hr**: hour (0 to 23)
* **holiday**: whether the day was a holiday or not
* **weekday**: day of the week
* **workingday**: `1` if the day is neither a weekend nor holiday, otherwise `0`.
* **weathersit**: 
  * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
  * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
  * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
  * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **temp**: Normalized temperature in Celsius. The values are derived via `(t-t_min)/(t_max-t_min)`, `t_min=-8`, `t_max=+39` (only in hourly scale)
* **atemp**: Normalized feeling temperature in Celsius. The values are derived via `(t-t_min)/(t_max-t_min)`, `t_min=-16`, `t_max=+50` (only in hourly scale)
* **hum**: Normalized humidity. The values are divided to 100 (max)
* **windspeed**: Normalized wind speed. The values are divided to 67 (max)

**Label columns**:
* **casual**: count of casual users
* **registered**: count of registered users
* **cnt**: count of total rental bikes including both casual and registered

**Extraneous columns**:
* **instant**: record index

For example, the first row is a record of hour 0 on January 1, 2011---and apparently, 16 people rented bikes around midnight!

## Preprocessing the data

So what do we need to do to get our data ready for Machine Learning?

**Recall our goal**: We want to learn to predict the count of bike rentals (the `cnt` column).  We refer to the count as our target "label".

**Features**: What can we use as features to predict the `cnt` label?  

All the columns except `cnt`, and a few exceptions:
* `casual` & `registered`
  * The `cnt` column we want to predict equals the sum of the `casual` + `registered` columns.  We will remove the `casual` and `registered` columns from the data to make sure we do not use them to predict `cnt`.  (*Warning: This is a danger in careless Machine Learning.  Make sure you do not "cheat" by using information you will not have when making predictions*)
* `season` and the date column `dteday`: We could keep them, but they are well-represented by the other date-related columns like `yr`, `mnth`, and `weekday`.
* `holiday` and `weekday`: These features are highly correlated with the `workingday` column.
* row index column `instant`: This is a useless column to us.

Let's drop the feature columns `instant`, `dteday`, `season`, `holiday`, `weekday`, and label columns `casual` and `registered` from our DataFrame and then review our schema:

In [12]:
preprocessedDF = initialDF.drop("instant", "dteday", "season", "holiday", "weekday", "casual", "registered")

preprocessedDF.printSchema()

root
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- cnt: integer (nullable = true)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Train/Test Split

Our final data preparation step will be to split our dataset into separate training and test sets.

Using the `randomSplit()` function, we split the data such that 70% of the data is reserved for training and the remaining 30% for testing. 

For more information see:
* Scala: <a href="https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.Dataset" target="_blank">Dataset.randomSplit()</a>
* Python: <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit" target="_blank">DataFrame.randomSplit()</a>

In [14]:
trainDF, testDF = preprocessedDF.randomSplit(
  [0.7, 0.3],  # 70-30 split
  seed=42)     # For reproducibility

print("We have %d training examples and %d test examples." % (trainDF.count(), testDF.count()))
assert (trainDF.count() == 12197)

We have 12197 training examples and 5182 test examples.

## Visualize our data

Now that we have preprocessed our features, we can quickly visualize our data to get a sense of whether the features are meaningful.

We want to compare bike rental counts versus the hour of the day. 

To plot the data:
* Run the cell below
* From the list of plot types, select **Line**.
* Click the **Plot Options...** button.
* By dragging and dropping the fields, set the **Keys** to **hr** and the **Values** to **cnt**.

Once you've created the graph, go back and select different **Keys**. For example:
* **cnt** vs. **windspeed**
* **cnt** vs. **month**
* **cnt** vs. **workingday**
* **cnt** vs. **hum**
* **cnt** vs. **temp**
* ...etc.

In [16]:
display(trainDF) #.limit(10))

yr,mnth,hr,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,0,1,0.04,0.0303,0.45,0.2537,13
0,1,0,0,1,0.18,0.2424,0.55,0.0,28
0,1,0,0,1,0.22,0.197,0.64,0.3582,28
0,1,0,0,1,0.24,0.2879,0.81,0.0,16
0,1,0,0,1,0.26,0.303,0.56,0.0,39
0,1,0,0,2,0.18,0.197,0.51,0.1642,25
0,1,0,0,2,0.2,0.197,0.47,0.2239,17
0,1,0,0,2,0.46,0.4545,0.88,0.2985,17
0,1,0,1,1,0.06,0.0606,0.41,0.194,7
0,1,0,1,1,0.12,0.1212,0.5,0.2836,5


A couple of notes:
* Rentals are low during the night, and they peak in the morning (8 am) and in the early evening (5 pm).  
* Rentals are high during the summer and low in winter.
* Rentals are high on working days vs. non-working days

This indicates that the `hr`, `mnth` and `workingday` features are all useful and can help us predict our label `cnt`. 

But how do other features affect our prediction? 

Do combinations of those features matter? For example, high wind in summer is not going to have the same effect as high wind in winter.

As it turns out our features can be divided into two types:
 * **Numeric columns:**
   * `mnth`
   * `temp`
   * `hr`
   * `hum`
   * `atemp`
   * `windspeed`

* **Categorical Columns:**
  * `yr`
  * `workingday`
  * `weathersit`
  
We could treat both `mnth` and `hr` as categorical but we would lose the temporal relationships (e.g. 2:00 AM comes before 3:00 AM).

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) StringIndexer

For each of the categorical columns, we are going to create one `StringIndexer` where we
  * Set `inputCol` to something like `weathersit`
  * Set `outputCol` to something like `weathersitIndex`

This will have the effect of treating a value like `weathersit` not as number 1 through 4, but rather four categories: **light**, **mist**, **medium** & **heavy**, for example.

For more information see:
* Scala: <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.feature.StringIndexer" target="_blank">StringIndexer</a>
* Python: <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=stringindexer#pyspark.ml.feature.StringIndexer" target="_blank">StringIndexer</a>

Before we get started, let's review our current schema:

In [20]:
trainDF.printSchema()

root
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- cnt: integer (nullable = true)

Let's create the first `StringIndexer` for the `workingday` column.

After we create it, we can run a sample through the indexer to see how it would affect our `DataFrame`.

In [22]:
from pyspark.ml.feature import StringIndexer

workingdayStringIndexer = StringIndexer(
  inputCol="workingday", 
  outputCol="workingdayIndex")

# Just for demonstration purposes, we will use the StringIndexer to fit and
# then transform our training data set just to see how it affects the schema
workingdayStringIndexer.fit(trainDF).transform(trainDF).printSchema()

root
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- cnt: integer (nullable = true)
-- workingdayIndex: double (nullable = false)

Next we will create the `StringIndexer` for the `yr` column and preview its effect.

In [24]:
yrStringIndexer = StringIndexer(
  inputCol="yr", 
  outputCol="yrIndex")

yrStringIndexer.fit(trainDF).transform(trainDF).printSchema()

root
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- cnt: integer (nullable = true)
-- yrIndex: double (nullable = false)

And then create our last `StringIndexer` for the `weathersit` column.

In [26]:
weathersitStringIndexer = StringIndexer(
  inputCol="weathersit", 
  outputCol="weathersitIndex")

weathersitStringIndexer.fit(trainDF).transform(trainDF).printSchema()

root
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- hr: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- cnt: integer (nullable = true)
-- weathersitIndex: double (nullable = false)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) VectorAssembler

The next step is to assemble the feature columns into a single feature vector.

To do that we will use the `VectorAssembler` where we
  * Set `inputCols` to the new list of feature columns
  * Set `outputCol` to `features`
  
  
For more information see:
* Scala: <a href="https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.ml.feature.VectorAssembler" target="_blank">VectorAssembler</a>
* Python: <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler" target="_blank">VectorAssembler</a>

In [28]:
from pyspark.ml.feature import VectorAssembler

assemblerInputs  = [
  "mnth", "temp", "hr", "hum", "atemp", "windspeed", # Our numerical features
  "yrIndex", "workingdayIndex", "weathersit"]        # Our new categorical features

vectorAssembler = VectorAssembler(
  inputCols=assemblerInputs, 
  outputCol="features")

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Random Forests

Random forests and ensembles of decision trees are more powerful than a single decision tree alone.

This is also the last step in our pipeline.

We will use the `RandomForestRegressor` where we
  * Set `labelCol` to the column that contains our label.
  * Set `seed` to ensure reproducibility.
  * Set `numTrees` to `3` so that we build 3 trees in our random forest.
  * Set `maxDepth` to `10` to control the depth/complexity of the tree.

For more information see:
* Scala: <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.regression.RandomForestRegressor" target="_blank">RandomForestRegressor</a>
* Python: <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor" target="_blank">RandomForestRegressor</a>

In [30]:
from pyspark.ml.regression import RandomForestRegressor

rfr = (RandomForestRegressor()
      .setLabelCol("cnt") # The column of our label
#      .setFeatureCols("features")
      .setSeed(27)        # Some seed value for consistency
      .setNumTrees(3)     # A guess at the number of trees
      .setMaxDepth(10)    # A guess at the depth of each tree
)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Create a Machine Learning Pipeline

Now let's wrap all of these stages into a Pipeline.

In [32]:
from pyspark.ml import Pipeline

pipeline = Pipeline().setStages([
  workingdayStringIndexer, # categorize workingday
  weathersitStringIndexer, # categorize weathersit
  yrStringIndexer,         # categorize yr
  vectorAssembler,         # assemble the feature vector for all columns
  rfr])

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Train the model

Train the pipeline model to run all the steps in the pipeline.

In [34]:
pipelineModel = pipeline.fit(trainDF)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Evaluate the model

Now that we have fitted a model, we can evaluate it.

In the case of a random forest, one of the best things to look at is the `featureImportances`:

In [36]:
from pyspark.ml.regression import RandomForestRegressionModel

rfrm = pipelineModel.stages[-1] # The RFRM is in the last stage of the model

#  Zip the list of features with their scores
scores = zip(assemblerInputs, rfrm.featureImportances)

# And pretty print 'em
for x in scores: print("%-15s = %s" % x)

print("-"*80)

mnth = 0.019951984999
temp = 0.0749611910561
hr = 0.532666491488
hum = 0.0488083805266
atemp = 0.121545867375
windspeed = 0.0124107551202
yrIndex = 0.0927445513793
workingdayIndex = 0.0806174757535
weathersit = 0.0162933023017
--------------------------------------------------------------------------------

Which features were most important?

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Making Predictions

Next, apply the trained pipeline model to the test set.

In [39]:
# Using the model, create our predictions from the test data
predictionsDF = pipelineModel.transform(testDF)

# Reorder the columns for easier interpretation
reorderedDF = predictionsDF.select("cnt", "prediction", "yr", "yrIndex", "mnth", "hr", "workingday", "workingdayIndex", "weathersit", "weathersitIndex", "temp", "atemp", "hum", "windspeed")

display(reorderedDF)

cnt,prediction,yr,yrIndex,mnth,hr,workingday,workingdayIndex,weathersit,weathersitIndex,temp,atemp,hum,windspeed
22,21.670602577499128,0,1.0,1,0,0,1.0,1,0.0,0.04,0.0758,0.57,0.1045
25,15.896464646464645,0,1.0,1,0,0,1.0,1,0.0,0.1,0.0758,0.42,0.3881
33,21.670602577499128,0,1.0,1,0,0,1.0,1,0.0,0.16,0.1818,0.8,0.1045
14,9.227377717391304,0,1.0,1,0,1,0.0,1,0.0,0.12,0.1364,0.5,0.194
5,12.633518714689265,0,1.0,1,0,1,0.0,1,0.0,0.16,0.1818,0.55,0.1045
3,12.557812499999999,0,1.0,1,0,1,0.0,2,1.0,0.22,0.2727,0.93,0.0
21,9.227377717391304,0,1.0,1,0,1,0.0,2,1.0,0.24,0.2273,0.7,0.2537
13,21.670602577499128,0,1.0,1,1,0,1.0,1,0.0,0.04,0.0758,0.57,0.1045
23,18.702020202020204,0,1.0,1,1,0,1.0,1,0.0,0.26,0.2727,0.56,0.1343
12,16.72979797979798,0,1.0,1,1,0,1.0,2,1.0,0.04,0.0,0.41,0.3881


##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Evaluate

Next, we'll use `RegressionEvaluator` to assess the results. The default regression metric is RMSE.

For more information see:
* Scala: <a href="https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.ml.evaluation.RegressionEvaluator" target="_blank">RegressionEvaluator</a>
* Python: <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator" target="_blank">RegressionEvaluator</a>

In [41]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator().setLabelCol("cnt")

rmse = evaluator.evaluate(predictionsDF)

print("Test RMSE = %f" % rmse)

Test RMSE = 72.037939

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) ParamGrid

There are a lot of hyperparamaters we could tune, and it would take a long time to manually configure.

Instead of a manual (ad-hoc) approach, let's use Spark's `ParamGridBuilder` to find the optimal hyperparameters in a more systematic approach.

In this example notebook, we keep these trees shallow and use a relatively small number of trees. Let's define a grid of hyperparameters to test:
  - maxDepth: max depth of each decision tree in the RF ensemble (Use the values `2, 5, 10`)
  - numTrees: number of trees in each RF ensemble (Use the values `10, 50`)

`addGrid()` accepts the name of the parameter (e.g. `rf.maxDepth`), and an Array of the possible values (e.g. `Array(2, 5, 10)`).

For more information see:
* Scala: <a href="https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.ml.tuning.ParamGridBuilder" target="_blank">ParamGridBuilder</a>
* Python: <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder" target="_blank">ParamGridBuilder</a>

In [43]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = (ParamGridBuilder()
            .addGrid(rfr.maxDepth, [2, 5, 10])
            .addGrid(rfr.numTrees, [10, 50])
            .build())

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Cross-Validation

We are also going to use 3-fold cross-validation to identify the optimal maxDepth and numTrees combination.

![crossValidation](https://files.training.databricks.com/images/301/CrossValidation.png)

With 3-fold cross-validation, we train on 2/3 of the data and evaluate with the remaining (held-out) 1/3. We repeat this process 3 times, so each fold gets the chance to act as the validation set. We then average the results of the three rounds.

We pass in the `estimator` (our original pipeline), an `evaluator`, and an `estimatorParamMaps` to the `CrossValidator` so that it knows:
- Which model to use
- How to evaluate the model
- What hyperparamters to set on the model

We can also set the number of folds we want to split our data into (3), as well as setting a seed so we all have the same split in the data.

For more information see:
* Scala: <a href="https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.ml.tuning.CrossValidator" target="_blank">CrossValidator</a>
* Python: <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator" target="_blank">CrossValidator</a>

In [46]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = (RegressionEvaluator()
  .setLabelCol("cnt")
  .setPredictionCol("prediction"))

cv = (CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3)
  .setSeed(27))

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) A New Model

We can now use the `CrossValidator` to fit a new model - this could take several minutes on a small cluster.

In [48]:
cvModel = cv.fit(trainDF)

And now we can take a look at the model with the best hyperparameter configuration:

In [50]:
# Zip the two lists together
results = list(zip(cvModel.getEstimatorParamMaps(), cvModel.avgMetrics))

# And pretty print 'em
for x in results:
  numTrees, rmse = list(x[0].values())
  print("Depth: %s, Trees: %s\nAverage: %s\n" % (numTrees, rmse, x[1]))
  
print("-"*80)

Depth: 2, Trees: 10
Average: 137.85945474074845

Depth: 2, Trees: 50
Average: 138.15147778221956

Depth: 5, Trees: 10
Average: 111.68513536907443

Depth: 5, Trees: 50
Average: 113.40621629592276

Depth: 10, Trees: 10
Average: 77.24611663592873

Depth: 10, Trees: 50
Average: 79.13960536813686

--------------------------------------------------------------------------------

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) One last set of predictions

Using our newest mode, let's make a final set of predictions:

In [52]:
# Using the model, create our predictions from the test data
finalPredictionsDF = cvModel.transform(testDF)

# Reorder the columns for easier interpretation
finalDF = finalPredictionsDF.select("cnt", "prediction", "yr", "yrIndex", "mnth", "hr", "workingday", "workingdayIndex", "weathersit", "weathersitIndex", "temp", "atemp", "hum", "windspeed")

display(finalDF)

cnt,prediction,yr,yrIndex,mnth,hr,workingday,workingdayIndex,weathersit,weathersitIndex,temp,atemp,hum,windspeed
22,25.723225957049486,0,1.0,1,0,0,1.0,1,0.0,0.04,0.0758,0.57,0.1045
25,19.490358324625568,0,1.0,1,0,0,1.0,1,0.0,0.1,0.0758,0.42,0.3881
33,23.406856909430438,0,1.0,1,0,0,1.0,1,0.0,0.16,0.1818,0.8,0.1045
14,12.233504469116564,0,1.0,1,0,1,0.0,1,0.0,0.12,0.1364,0.5,0.194
5,13.31906002467212,0,1.0,1,0,1,0.0,1,0.0,0.16,0.1818,0.55,0.1045
3,16.658668394988503,0,1.0,1,0,1,0.0,2,1.0,0.22,0.2727,0.93,0.0
21,19.50940687219757,0,1.0,1,0,1,0.0,2,1.0,0.24,0.2273,0.7,0.2537
13,21.758578521152053,0,1.0,1,1,0,1.0,1,0.0,0.04,0.0758,0.57,0.1045
23,17.25894014074161,0,1.0,1,1,0,1.0,1,0.0,0.26,0.2727,0.56,0.1343
12,17.584589093856334,0,1.0,1,1,0,1.0,2,1.0,0.04,0.0,0.41,0.3881


##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Evaluating the New Model

Let's see how our latest model does:

In [54]:
print("Test RMSE = %f" % evaluator.evaluate(finalPredictionsDF))

Test RMSE = 74.524012

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>